In [1]:
!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 torchtext==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

!pip install spacy
!python -m spacy download en_core_web_sm


import torch
from torchtext.legacy import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm')
LABEL = data.LabelField(dtype = torch.float)

from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')
print(vars(train_data.examples[0]))

import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")
print(TEXT.vocab.freqs.most_common(20))
print(TEXT.vocab.itos[:10])
print(LABEL.vocab.stoi)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Do not use BucketIterator in your implementation because you are required to implement the padding and masking yourself.
# train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits((train_data, valid_data, test_data), batch_size=1, device=device)

from torch.utils.data import DataLoader, TensorDataset

# 先將數據轉為張量
def process_data(data, text_field, label_field):
    text_tensors = [torch.tensor(text_field.process([vars(example)['text']])) for example in data]
    label_tensors = [torch.tensor(label_field.process([vars(example)['label']]), dtype=torch.float) for example in data]
    
    # Padding text sequences to the same length
    text_tensors = torch.nn.utils.rnn.pad_sequence(text_tensors, batch_first=True, padding_value=1)

    dataset = TensorDataset(text_tensors, torch.cat(label_tensors))
    return dataset

train_dataset = process_data(train_data, TEXT, LABEL)
valid_dataset = process_data(valid_data, TEXT, LABEL)
test_dataset = process_data(test_data, TEXT, LABEL)

# 建立 DataLoader 進行批次處理
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


import torch.nn as nn
class LR(nn.Module):
    def __init__(self, input_dim, embedding_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx=1)  # 設置 padding_idx 避免影響
        self.fc = nn.Linear(embedding_dim, output_dim)

    def forward(self, text):
        embedded = self.embedding(text)  # (batch_size, seq_len, embedding_dim)
        embedded = embedded.mean(dim=1)  # 對所有時間步取平均
        return self.fc(embedded)

INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
OUTPUT_DIM = 1

model = LR(INPUT_DIM, EMBEDDING_DIM, OUTPUT_DIM)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

def binary_accuracy(preds, y):
    """
    Returns accuracy per batch.
    """
    # 將 preds 經過 sigmoid 並四捨五入為 0 或 1
    rounded_preds = torch.round(torch.sigmoid(preds))  # [batch_size, 1]

    # 確保 y 的形狀與 rounded_preds 相同S
    y = y.unsqueeze(1) if len(y.shape) == 1 else y

    # 比較預測與真實標籤，並轉換為浮點數
    correct = (rounded_preds == y).float()

    # 計算準確率
    acc = correct.sum() / correct.numel()
    return acc


from tqdm import tqdm
def train(model, loader, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.train()

    for batch in loader:
        text, labels = batch
        text, labels = text.to(device), labels.to(device)

        optimizer.zero_grad()
        predictions = model(text).squeeze(1)  # 模型輸出，形狀為 [batch_size, 1]

        loss = criterion(predictions, labels.unsqueeze(1))  # 確保形狀匹配
        acc = binary_accuracy(predictions, labels)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(loader), epoch_acc / len(loader)


def evaluate(model, loader, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()

    with torch.no_grad():
        for batch in loader:
            text, labels = batch
            text, labels = text.to(device), labels.to(device)

            predictions = model(text).squeeze(1)

            loss = criterion(predictions, labels.unsqueeze(1))

            acc = binary_accuracy(predictions, labels)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(loader), epoch_acc / len(loader)

import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

N_EPOCHS = 5
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()

    train_loss, train_acc = train(model, train_loader, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_loader, criterion)

    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'best-model.pt')

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')


model.load_state_dict(torch.load('best-model.pt'))
test_loss, test_acc = evaluate(model, test_loader, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')



Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ----------------------------------- --- 11.8/12.8 MB 56.8 MB/s eta 0:00:01
     --------------------------------------- 12.8/12.8 MB 50.2 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Number of training examples: 25000
Number of testing examples: 25000
{'text': ['Bromwell', 'High', 'is', 'a', 'cartoon', 'comedy', '.', 'It', 'ran', 'at', 'the', 'same', 'time', 'as', 'some', 'other', 'programs', 'about', 'school', 'life', ',', 'such', 'as', '"', 'Teachers', '"', '.', 'My', '35', 'years', 'in', 'the', 'teaching', 'profession', 'lead', 'me', 'to', 'believe', 'that', 'Bromwell', 'High', "'s", 'satire', 'is', 'much', 'closer', 'to', 'reality', 'than', 'is', '"', 'Teachers', '"', '.', 'The', 'scramble', 'to', 'survive', 'financially', ',', 'the', 'insightful', 'students', 'who

C:\Users\USER\AppData\Local\Temp\ipykernel_26752\2875315650.py:50: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  text_tensors = [torch.tensor(text_field.process([vars(example)['text']])) for example in data]
C:\Users\USER\AppData\Local\Temp\ipykernel_26752\2875315650.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label_tensors = [torch.tensor(label_field.process([vars(example)['label']]), dtype=torch.float) for example in data]


The model has 2,500,301 trainable parameters
Epoch: 01 | Time: 0m 4s
	Train Loss: 0.694 | Train Acc: 49.67%
	 Val. Loss: 0.693 |  Val. Acc: 50.87%
Epoch: 02 | Time: 0m 2s
	Train Loss: 0.694 | Train Acc: 49.66%
	 Val. Loss: 0.693 |  Val. Acc: 50.91%
Epoch: 03 | Time: 0m 2s
	Train Loss: 0.694 | Train Acc: 49.67%
	 Val. Loss: 0.693 |  Val. Acc: 50.91%
Epoch: 04 | Time: 0m 2s
	Train Loss: 0.694 | Train Acc: 49.64%
	 Val. Loss: 0.693 |  Val. Acc: 50.81%
Epoch: 05 | Time: 0m 2s
	Train Loss: 0.694 | Train Acc: 49.69%
	 Val. Loss: 0.693 |  Val. Acc: 50.81%
Test Loss: 0.693 | Test Acc: 49.89%


In [2]:
# 單一樣本
single_text, single_label = train_dataset[0]
single_text = single_text.unsqueeze(0).to(device)  # 添加 batch 維度
single_label = single_label.unsqueeze(0).to(device)

# 批次中的樣本
batch_text, batch_labels = next(iter(train_loader))
batch_text, batch_labels = batch_text.to(device), batch_labels.to(device)

# 計算分數
with torch.no_grad():
    single_pred = model(single_text).squeeze(1)
    batch_preds = model(batch_text).squeeze(1)

# 比較單一輸出與批次中的第一個樣本
print(f"Single instance score: {single_pred.item()}")
print(f"Batch instance score: {batch_preds[0].item()}")


Single instance score: 0.05038141831755638
Batch instance score: 0.05061117559671402


In [22]:
import pandas as pd

predictions = []
model.eval()
with torch.no_grad():
    for batch in test_loader:
        text, _ = batch
        text = text.to(device)
        preds = torch.sigmoid(model(text)).cpu().numpy()
        predictions.extend(preds)

df = pd.DataFrame({'predictions': predictions})
df.to_csv('test_predictions.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt
import time

batch_sizes = [4]
training_times = []

for batch_size in batch_sizes:
    # 更新 DataLoader
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # 記錄訓練時間
    start_time = time.time()
    train_loss, train_acc = train(model, train_loader, optimizer, criterion)
    end_time = time.time()

    training_times.append(end_time - start_time)

# 畫圖
plt.plot(batch_sizes, training_times)
plt.xlabel('Batch Size')
plt.ylabel('Training Time (seconds)')
plt.title('Training Time vs Batch Size')
plt.show()


In [4]:
accuracies = []

for batch_size in batch_sizes:
    # 更新 DataLoader
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

    # 訓練並驗證
    train_loss, train_acc = train(model, train_loader, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_loader, criterion)
    accuracies.append(valid_acc)

# 畫圖
plt.plot(batch_sizes, accuracies)
plt.xlabel('Batch Size')
plt.ylabel('Validation Accuracy')
plt.title('Validation Accuracy vs Batch Size')
plt.show()


NameError: name 'batch_sizes' is not defined

In [2]:
import torch
from torchtext.legacy import data
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
import time

# 固定隨機種子
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# 定義 Field 和數據集
TEXT = data.Field(tokenize='spacy', tokenizer_language='en_core_web_sm')
LABEL = data.LabelField(dtype=torch.float)

from torchtext.legacy import datasets
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
import random
train_data, valid_data = train_data.split(random_state=random.seed(SEED))


MAX_VOCAB_SIZE = 25_000
TEXT.build_vocab(train_data, max_size=MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')







C:\Users\USER\AppData\Local\Temp\ipykernel_11316\1517062520.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  text_tensors.append(torch.tensor(text_field.process([tokens])))
C:\Users\USER\AppData\Local\Temp\ipykernel_11316\1517062520.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label_tensors.append(torch.tensor(label_field.process([vars(example)['label']]), dtype=torch.float))


The model has 2,867,049 trainable parameters


RuntimeError: input must have 2 dimensions, got 3

In [ ]:
from torch.nn.utils.rnn import pad_sequence

def process_data(data, text_field, label_field):
    text_tensors = []
    label_tensors = []
    text_lengths = []

    for example in data:
        tokens = vars(example)['text']
        text_tensors.append(torch.tensor(text_field.process([tokens])))
        text_lengths.append(len(tokens))
        label_tensors.append(torch.tensor(label_field.process([vars(example)['label']]), dtype=torch.float))

    text_tensors = pad_sequence(text_tensors, batch_first=True, padding_value=1)
    dataset = TensorDataset(text_tensors, torch.cat(label_tensors), torch.tensor(text_lengths))
    return dataset

train_dataset = process_data(train_data, TEXT, LABEL)
valid_dataset = process_data(valid_data, TEXT, LABEL)
test_dataset = process_data(test_data, TEXT, LABEL)


In [ ]:
# 定義 LSTM 模型
class LSTMModel(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, padding_idx):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx=padding_idx)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, text, text_lengths):
        # text: [batch_size, seq_len]
        embedded = self.embedding(text)  # [batch_size, seq_len, embedding_dim]
    
        # Pack the sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(
            embedded, text_lengths.cpu(), batch_first=True, enforce_sorted=False
        )
    
        # LSTM
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
    
        # Unpack the sequence
        output, _ = nn.utils.rnn.pad_packed_sequence(packed_output, batch_first=True)
    
        # Average pooling
        hidden_mean = output.mean(dim=1)  # [batch_size, hidden_dim]
        
        return self.fc(hidden_mean)  # [batch_size, output_dim]


# 初始化模型
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
PADDING_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = LSTMModel(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, PADDING_IDX)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {count_parameters(model):,} trainable parameters')


In [ ]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

# 定義準確率計算函數
def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    return correct.sum() / len(y)

# 訓練函數
def train(model, loader, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    
    for batch in loader:
        text, labels, text_lengths = batch
        text, labels, text_lengths = text.to(device), labels.to(device), text_lengths.to(device)

        optimizer.zero_grad()
        predictions = model(text, text_lengths).squeeze(1)
        loss = criterion(predictions, labels)
        acc = binary_accuracy(predictions, labels)
        
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()
    
    return epoch_loss / len(loader), epoch_acc / len(loader)

# 評估函數
def evaluate(model, loader, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    
    with torch.no_grad():
        for batch in loader:
            text, labels, text_lengths = batch
            text, labels, text_lengths = text.to(device), labels.to(device), text_lengths.to(device)

            predictions = model(text, text_lengths).squeeze(1)
            loss = criterion(predictions, labels)
            acc = binary_accuracy(predictions, labels)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
    
    return epoch_loss / len(loader), epoch_acc / len(loader)

# 計時函數
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - elapsed_mins * 60)
    return elapsed_mins, elapsed_secs

# 訓練模型
N_EPOCHS = 5
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_loader, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_loader, criterion)
    
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'best-lstm-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

# 測試模型
model.load_state_dict(torch.load('best-lstm-model.pt'))
test_loss, test_acc = evaluate(model, test_loader, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

In [3]:
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.utils.rnn import pad_sequence
import torch.nn as nn
import torch.optim as optim
import time

# 固定隨機種子
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# 定義 Field
TEXT = data.Field(tokenize='spacy', tokenizer_language='en_core_web_sm')
LABEL = data.LabelField(dtype=torch.float)

# 加載數據集
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
train_data, valid_data = train_data.split(random_state=torch.manual_seed(SEED))

# 構建詞彙表
MAX_VOCAB_SIZE = 25_000
TEXT.build_vocab(train_data, max_size=MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 數據處理函數
def process_data(data, text_field, label_field):
    text_tensors = []
    label_tensors = []
    text_lengths = []

    for example in data:
        tokens = vars(example)['text']
        text_tensors.append(torch.tensor(text_field.process([tokens])))
        text_lengths.append(len(tokens))
        label_tensors.append(torch.tensor(label_field.process([vars(example)['label']]), dtype=torch.float))

    text_tensors = pad_sequence(text_tensors, batch_first=True, padding_value=1)
    dataset = TensorDataset(text_tensors, torch.cat(label_tensors), torch.tensor(text_lengths))
    return dataset

# 數據集和 DataLoader
train_dataset = process_data(train_data, TEXT, LABEL)
valid_dataset = process_data(valid_data, TEXT, LABEL)
test_dataset = process_data(test_data, TEXT, LABEL)

# 自定義 collate_fn 確保排序
def collate_fn(batch):
    batch.sort(key=lambda x: x[2], reverse=True)  # 按句子長度排序
    text, labels, text_lengths = zip(*batch)
    text = torch.stack(text)
    labels = torch.stack(labels)
    text_lengths = torch.tensor(text_lengths)
    return text, labels, text_lengths

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, collate_fn=collate_fn)

# 定義 LSTM 模型
class LSTMModel(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, padding_idx):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx=padding_idx)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, text, text_lengths):
        embedded = self.embedding(text)  # (batch_size, seq_len, embedding_dim)
        packed_embedded = nn.utils.rnn.pack_padded_sequence(
            embedded, text_lengths.cpu(), batch_first=True, enforce_sorted=False
        )
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        output, _ = nn.utils.rnn.pad_packed_sequence(packed_output, batch_first=True)
        hidden_mean = output.mean(dim=1)  # (batch_size, hidden_dim)
        return self.fc(hidden_mean)

# 初始化模型
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
PADDING_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = LSTMModel(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, PADDING_IDX)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {count_parameters(model):,} trainable parameters')

# 設置優化器和損失函數
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

# 準確率計算函數
def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    return correct.sum() / len(y)

# 訓練函數
def train(model, loader, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    
    for batch in loader:
        text, labels, text_lengths = batch
        text, labels, text_lengths = text.to(device), labels.to(device), text_lengths.to(device)

        optimizer.zero_grad()
        predictions = model(text, text_lengths).squeeze(1)
        loss = criterion(predictions, labels)
        acc = binary_accuracy(predictions, labels)
        
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()
    
    return epoch_loss / len(loader), epoch_acc / len(loader)

# 評估函數
def evaluate(model, loader, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    
    with torch.no_grad():
        for batch in loader:
            text, labels, text_lengths = batch
            text, labels, text_lengths = text.to(device), labels.to(device), text_lengths.to(device)

            predictions = model(text, text_lengths).squeeze(1)
            loss = criterion(predictions, labels)
            acc = binary_accuracy(predictions, labels)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
    
    return epoch_loss / len(loader), epoch_acc / len(loader)

# 計時函數
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - elapsed_mins * 60)
    return elapsed_mins, elapsed_secs

# 訓練模型
N_EPOCHS = 5
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_loader, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_loader, criterion)
    
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'best-lstm-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

# 測試模型
model.load_state_dict(torch.load('best-lstm-model.pt'))
test_loss, test_acc = evaluate(model, test_loader, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')


TypeError: 'torch._C.Generator' object is not subscriptable